### <span style=color:blue> Loading Listings & Reviews data from postgresql into local MongoDB    </span>

In [49]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

import datetime
from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('helper_functions/')
# import util as util
import util

<span style=color:blue>Getting access to postgres set up</span>

In [12]:
# Load the env file 

dotenv_path = 'env_variables.env'
load_dotenv(dotenv_path=dotenv_path)

# Import the env variables

load_dotenv()

schema = os.getenv('DISC_6_SCHEMA')
port = os.getenv('DISC_6_PORT')
host = os.getenv('DISC_6_HOST')
database = os.getenv('DISC_6_DB')
password = os.getenv('DISC_6_PASSWORD')
connection = os.getenv('DISC_6_CONNECTION')

# Create the db engine 

db_eng = create_engine(f"postgresql+psycopg2://{connection}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(schema)},
                       isolation_level = 'SERIALIZABLE')

print("Successfully created db engine.")

Successfully created db engine.


<span style=color:blue>Getting mongodb connection set up</span>

In [3]:
from pymongo import MongoClient

client = MongoClient()
# could have written client = MongoClient("localhost", 27017)
#                 or client = MongoClient("mongodb://localhost:27017/")

<span style=color:blue>Getting access to mongodb airbnb database, and setting up collection "cal" to hold the calendar data in mongodb</span>

In [4]:
# I have (or will have) a database "airbnb"
db = client.airbnb

# inside the "airbnb" database, I have (or will have) a collection "listings"
cal = db.calendar

print('The list of all databases currently in the MongoDB client is:')
print(client.list_database_names())

print('\nThe list of all collections in the airbnb database is:')
print(db.list_collection_names())
# Note: calendar may not show up yet; it is created only when a first document is inserted into it

The list of all databases currently in the MongoDB client is:
['admin', 'airbnb', 'config', 'local', 'test']

The list of all collections in the airbnb database is:
['testing', 'listings']


 <span style=color:blue> Fetching list of all listing_ids    </span>

In [30]:
importlib.reload(util)

q_listing_ids = util.build_query_get_all_listing_ids()
print(q_listing_ids)

print()

listing_id = 'test'
q_cal_selection = util.build_query_calendar_for_listing(listing_id)
print(q_cal_selection)

with db_eng.connect() as conn:
    df_l = pd.read_sql(q_listing_ids, con=conn)

print()
print(df_l.shape)
print(df_l.head(20))


select distinct id as listing_id
from listings
order by id

select date, available, price,
       adjusted_price, minimum_nights, maximum_nights
from calendar
where listing_id = 'test'

(39202, 1)
             listing_id
0              10000070
1                100002
2   1000023148920376325
3   1000027657512767160
4   1000041661179181846
5              10001022
6   1000103124244467969
7   1000114407765528259
8   1000126313762418098
9   1000128045386999700
10  1000140168422601037
11  1000155970237732964
12  1000159308811483583
13  1000190119235265402
14  1000280166040727910
15  1000321909937207752
16  1000329137658400675
17  1000342028803764294
18  1000350053844887112
19  1000368307266195832


In [27]:
dict_l = df_l.to_dict('records')

for i in range(0,10):
    print(dict_l[i])

{'listing_id': '10000070'}
{'listing_id': '100002'}
{'listing_id': '1000023148920376325'}
{'listing_id': '1000027657512767160'}
{'listing_id': '1000041661179181846'}
{'listing_id': '10001022'}
{'listing_id': '1000103124244467969'}
{'listing_id': '1000114407765528259'}
{'listing_id': '1000126313762418098'}
{'listing_id': '1000128045386999700'}


In [34]:
# also converts NaT to None, because MongoDB does not recognize NaT
def convert_date_str_to_datetime(dt):
    print(dt)
    if dt is None:
        return None
    elif pd.isnull(dt):  # tests whether dt is the pandas value NaT ("not a time")
        # print('\nEntered the NaT case\n')
        return None
    elif dt != dt:
        return None        # could also use math.nan, I think
    elif len(dt) != 10:
        print(dt)
        return None
    else:
        year = int(dt[0:4])
        month = int(dt[5:7])
        day = int(dt[8:10])
        # print(year, month, day)
        temp = datetime(year, month, day)
        ts = temp.timestamp()
        new_dt = datetime.fromtimestamp(ts)
        return new_dt

print(convert_date_str_to_datetime('2024-02-06'))


2024-02-06
2024-02-06 00:00:00


<span style=color:blue>Loop to fetch, for each listing_id, the set of calendar events related to that listing_id.  Using database fetches, so there will be roughly 39K of them </span>

In [52]:
i = 0

time1 = datetime.now()

for d in dict_l:
# for d in dict_l;
    i += 1

    listing_id = d['listing_id']

    q_cal_selection = util.build_query_calendar_for_listing(listing_id)

    with db_eng.connect() as conn:
        df_c = pd.read_sql(q_cal_selection, con=conn)

    # print(type(df_c.loc[0,'date']))

    # df_c['date'] = df_c['date'].apply(convert_date_str_to_datetime)
    df_c['date'] = df_c['date'].apply(pd.to_datetime)
    df_c['available'] = df_c['available'].apply(convert_tf_to_boolean)
    df['price'] = df['price'].apply(lambda x:x.replace('$','')) 

    dict_c = df_c.to_dict('records')

    # pprint.pp(dict_c)

    # print(df_cal_one_listing.head(30))

    d['dates_info'] = dict_c

    if i % 50 == 0:
        time2 = datetime.now()
        time_taken = util.time_diff(time1,time2)
        print('Have now completed step number:', str(i), 'and it took', str(time_taken), 'seconds' )
        time1 = datetime.now()

print(len(dict_l))
pprint.pp(dict_l[-5:-1])
# pprint.pp(dict_new[-5:-1])


Have now completed step number: 50 and it took 132.750489 seconds
Have now completed step number: 100 and it took 142.902647 seconds
Have now completed step number: 150 and it took 133.645908 seconds
Have now completed step number: 200 and it took 157.889157 seconds
Have now completed step number: 250 and it took 155.190546 seconds
Have now completed step number: 300 and it took 150.734829 seconds
Have now completed step number: 350 and it took 160.137809 seconds
Have now completed step number: 400 and it took 162.796433 seconds
Have now completed step number: 450 and it took 153.107498 seconds
Have now completed step number: 500 and it took 147.444742 seconds
Have now completed step number: 550 and it took 151.962157 seconds
Have now completed step number: 600 and it took 161.159489 seconds
Have now completed step number: 650 and it took 160.965842 seconds
Have now completed step number: 700 and it took 144.054673 seconds
Have now completed step number: 750 and it took 179.892333 seco

KeyboardInterrupt: 

In [51]:
print(datetime.now())

2024-05-24 18:43:06.629567


<span style=color:blue>FAILED ATTEMPT AT: Loading contents of calendar csv file into a dataframe</span>

In [5]:
"""
# This became a black hole!

filename = '/Users/rick/DM-for-DS--Davis-Spring-2024/DATA-SETS/AirBNB/New-York-City/calendar.csv'

# see https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options for
#    various info about including data types into the read_csv command
# see also https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes
# As for the date and available fields (intended as date type and boolean, respectively,
#    we import as strings and convert in the data frame
dtype = {"listing_id": str, "date": str, "available": str, 
        "price": str, "adjusted_price": str, "minimum_nights": int, "maximum_nights": int}

# the csv has nulls in "adjusted_price", which has type str,. so including keep_default_na=False, 
#    see https://stackoverflow.com/questions/10867028/get-pandas-read-csv-to-read-empty-values-as-empty-string-instead-of-nan

# there are 7000+ entries for minimum_nights and maximum_nights with NULL values
# so following this pattern:
#    df2 = pd.read_csv('test_int64.csv', converters={'nr_nan':lambda x: pd.NA if x == '' else int(x)})
#    df2.nr_nan = df2.nr_nan.astype('Int64')
# see https://stackoverflow.com/questions/70776957/pandas-read-csv-with-integer-columns-with-null-values-without-precision-loss
converters = {'nr_nan':lambda x: pd.NA if x == '' else int(x)}

df = pd.read_csv(filename, dtype=dtype, converters=converters, keep_default_na=False)
df.nr_nan = df.nr_nan.astype('Int64')

print('The datatypes for the fields of df are:')
print(df.types)

print('\nThe first few rows of df are:')
print(df.head())
"""
print('This was a black hole')

This was a black hole


<span style=color:blue>Loading contents of calendar csv file into a dataframe</span>

<span style=color:blue>The system will give a warning, but it appears safe to ignore it.</span>

In [6]:
filename = '/Users/rick/DM-for-DS--Davis-Spring-2024/DATA-SETS/AirBNB/New-York-City/calendar.csv'

# Using partial list of dtypes, so that first several fields are interpreted as strings
# As for the date and available fields (intended as date type and boolean, respectively,
#    we import as strings and convert in the data frame
dtype = {"listing_id": str, "date": str, "available": str, 
        "price": str, "adjusted_price": str}
# note including these, because the null values make trouble:  , "minimum_nights": int, "maximum_nights": int}

# the csv has nulls in "adjusted_price", which has type str,. so including keep_default_na=False, 
#    see https://stackoverflow.com/questions/10867028/get-pandas-read-csv-to-read-empty-values-as-empty-string-instead-of-nan
         
df = pd.read_csv(filename, dtype=dtype, keep_default_na=False)

/var/folders/3l/gd1qj_mw8xl3gm001s6w9h180000gp/T/ipykernel_75892/2422796389.py:13: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, dtype=dtype, keep_default_na=False)


In [7]:
print('The datatypes for the fields of df are:')
print(df.dtypes)

print('\nThe first few rows of df are:')
print(df.head())

The datatypes for the fields of df are:
listing_id        object
date              object
available         object
price             object
adjusted_price    object
minimum_nights    object
maximum_nights    object
dtype: object

The first few rows of df are:
  listing_id        date available    price adjusted_price minimum_nights  \
0     144087  2024-02-10         t  $259.00                            30   
1     144087  2024-02-11         t  $259.00                            30   
2     144087  2024-02-12         t  $259.00                            30   
3     144087  2024-02-13         t  $259.00                            30   
4     144087  2024-02-14         t  $259.00                            30   

  maximum_nights  
0            365  
1            365  
2            365  
3            365  
4            365  


<span style=color:blue>Function to convert dates into datetimes.  This is useful because MongoDB recognizes datetime types but not date types.  So, will convert all dates into datetimes for insertion into MongoDB (after which we can do date arithmetic).</span>

<span style=color:blue>It also has conditions that turn various kinds of null values into None.  (Note: curiously this works on small dataframes, but left some values of "NaT" when applied on very large dataframes.)</span>

In [8]:
# also converts NaT to None, because MongoDB does not recognize NaT
def convert_date_str_to_datetime(dt):
    if dt is None:
        return None
    elif pd.isnull(dt):  # tests whether dt is the pandas value NaT ("not a time")
        # print('\nEntered the NaT case\n')
        return None
    elif dt != dt:
        return None        # could also use math.nan, I think
    elif dt == '':
        return None
    else:
        year = int(dt[0:4])
        month = int(dt[5:7])
        day = int(dt[8:10])
        # print(year, month, day)
        temp = datetime(year, month, day)
        ts = temp.timestamp()
        new_dt = datetime.fromtimestamp(ts)
        return new_dt

print(convert_date_str_to_datetime('2024-05-23'))


2024-05-23 00:00:00


<span style=color:blue>Function to convert the values of field "available" to booleans </span>

In [9]:
def convert_tf_to_boolean(val):
    if val == 't':
        return True
    elif val == 'f':
        return False
    else:
        return None

print(convert_tf_to_boolean('t'), convert_tf_to_boolean('f'), convert_tf_to_boolean('foo'))


True False None


<span style=color:blue>Cleaning up the values in df, to be more compatible with MongoDB  </span>

In [10]:
df['date'] = df['date'].apply(convert_date_str_to_datetime)
df['available'] = df['available'].apply(convert_tf_to_boolean)

print(df.head())

  listing_id       date  available    price adjusted_price minimum_nights  \
0     144087 2024-02-10       True  $259.00                            30   
1     144087 2024-02-11       True  $259.00                            30   
2     144087 2024-02-12       True  $259.00                            30   
3     144087 2024-02-13       True  $259.00                            30   
4     144087 2024-02-14       True  $259.00                            30   

  maximum_nights  
0            365  
1            365  
2            365  
3            365  
4            365  


In [11]:
df['price'] = df['price'].apply(lambda x:x.replace('$','')) 

print(df.head())

  listing_id       date  available   price adjusted_price minimum_nights  \
0     144087 2024-02-10       True  259.00                            30   
1     144087 2024-02-11       True  259.00                            30   
2     144087 2024-02-12       True  259.00                            30   
3     144087 2024-02-13       True  259.00                            30   
4     144087 2024-02-14       True  259.00                            30   

  maximum_nights  
0            365  
1            365  
2            365  
3            365  
4            365  


### <span style=color:blue>Now working to create a list of documents, where each document has a listing_id and then a list of dictionaries about available dates for that listing  </span>

<span style=color:blue>First step is to get a dataframe with just the listing_ids</span>

In [45]:
cols = df.columns.tolist()
print(cols)

['listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights']


In [46]:
other_cols = ['date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights']

df_new = df.drop(other_cols, axis=1).drop_duplicates()

print(df.shape)
print(df_new.shape)
print(df_new.head())


(14299870, 7)
(39201, 1)
     listing_id
0        144087
152        2595
322       43105
507      144148
1310      44229


<span style=color:blue>Next, convert df_new into a list of dictionaries</span>

In [48]:
# I'm calling it "dict_new" but actually it is a list of dictionaries
dict_new = df_new.to_dict('records')

pprint.pp(dict_new[0:10])

[{'listing_id': '144087'},
 {'listing_id': '2595'},
 {'listing_id': '43105'},
 {'listing_id': '144148'},
 {'listing_id': '44229'},
 {'listing_id': '5121'},
 {'listing_id': '146726'},
 {'listing_id': '6848'},
 {'listing_id': '44973'},
 {'listing_id': '6872'}]


<span style=color:blue>Now adding in all of the dates for each listing.  This might take an hour or more to run so plan accordingly</span>

In [ ]:
i = 0

# for d in dict_new[0:2]:
for d in dict_new:
    i += 1

    # print(d['listing_id'])

    df_cal_one_listing = df.loc[df['listing_id'] == d['listing_id']].drop(['listing_id'], axis=1)

    # print(df_cal_one_listing.head(30))

    dicts_cal_one_listing = df_cal_one_listing.to_dict('records')

    d['dates_info'] = dicts_cal_one_listing

    if i % 50 == 0:
        print('Have now completed step number:', str(i))

# print(len(dict_new))
pprint.pp(dict_new[-5:-1])


Have now completed step number: 50
Have now completed step number: 100
Have now completed step number: 150
Have now completed step number: 200
Have now completed step number: 250
Have now completed step number: 300
Have now completed step number: 350
Have now completed step number: 400
Have now completed step number: 450
Have now completed step number: 500
Have now completed step number: 550
Have now completed step number: 600
Have now completed step number: 650
Have now completed step number: 700
Have now completed step number: 750
Have now completed step number: 800
Have now completed step number: 850
Have now completed step number: 900
Have now completed step number: 950
Have now completed step number: 1000
Have now completed step number: 1050
Have now completed step number: 1100
Have now completed step number: 1150
Have now completed step number: 1200
Have now completed step number: 1250
Have now completed step number: 1300
Have now completed step number: 1350
Have now completed st

<span style=color:blue>First step is to get a dataframe with just the listing_ids</span>